<a href="https://colab.research.google.com/github/bee-llel/Machine-Learning/blob/master/RecurrentNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [144]:
#librairy:
import pandas as pd # used to manipulate the data 
import numpy as np
import scipy as sp 
from numpy import *


In [145]:
import random as rd 
from random import *
#allows us to always have the same result when calling a random function
np.random.seed(23)
# this small part was made with another group in class ( it's the lab) so it's the same as a few classmate.

#######################################
# DATA : random binary number 

X_train = []
X_test = [] 
y_train = []
y_test = [] 

for i in range(30) :
  X = [randint(0, 1) for i in range(10) ]
  X_train.append(X)
  y_train.append(sum(X))

for i in range(10):
  X = [randint(0, 1) for i in range(10) ]
  X_test.append(X)
  y_test.append(sum(X))

X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

y_train.reshape((30,1))
print(X_train)
print(y_train)
print(y_train.shape)

[[0 1 0 0 1 0 0 0 0 1]
 [1 1 0 0 0 1 0 0 1 0]
 [1 0 1 0 0 1 0 1 1 1]
 [0 0 0 0 0 1 0 0 0 1]
 [1 0 1 0 1 1 1 0 0 1]
 [0 1 0 0 1 1 1 1 1 0]
 [0 0 0 0 0 1 1 0 0 0]
 [1 0 1 0 1 1 1 1 0 1]
 [0 0 0 0 1 1 1 0 1 0]
 [1 0 0 1 0 1 1 0 1 0]
 [0 1 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 1 1 1 0 0]
 [1 1 1 1 0 1 0 1 0 1]
 [0 1 0 1 0 1 1 1 1 0]
 [0 1 0 1 0 0 0 1 0 0]
 [0 0 1 1 1 0 1 0 0 1]
 [1 1 0 1 1 1 0 1 0 0]
 [1 0 1 1 0 1 1 0 0 1]
 [1 1 0 1 0 0 1 0 1 1]
 [1 1 0 1 0 1 1 0 1 1]
 [1 0 0 0 0 1 1 1 0 0]
 [1 0 1 0 0 0 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 1]
 [0 1 1 0 0 0 0 0 0 1]
 [0 1 1 1 1 1 1 0 0 1]
 [1 0 1 0 0 1 0 1 0 1]
 [0 0 1 0 0 1 1 1 0 0]
 [0 1 0 0 0 1 0 1 0 1]
 [0 1 0 1 0 1 1 1 1 1]
 [1 0 0 0 1 0 1 1 0 1]]
[3 4 6 2 6 6 2 7 4 5 2 4 7 6 3 5 6 6 6 7 4 2 9 3 7 5 4 4 7 5]
(30,)


In [146]:
#################################
######################################
# VARIABLE DECLARATION:
###################################################
#basic number 
#T = lenght of the sequence 
T = 10
#
#I = data size 
I = 30
#
#J = output dimension = 1 not needed
J = 1
#
#K = number of hidden neuron  not needed
###################################################
# data: imput X = X_train and output y =  y_train:
# dimension : X  : I * T

X = X_train

# dimension : y : I * J = 30*1

y = y_train

###################################################
# activation function F = identity 

# dimension : same as X i think 
# we will initialize it's value to one to start 
F = np.zeros((I,T))
#our estimation G is the last F is the sequence this time since there is no activation function
#we will initialize it to zero.

###################
# G for backprop
G = np.zeros((I))

###################################################
#Weight:
# Vf: weight of F 
#dimension : scalar
# Vx:  weight of X 
#dimension : scalar

# weight for the backprop
Vxbp = np.random.uniform(0, 1)*0.001
Vfbp = np.random.uniform(0, 1)*0.001

# weight for the resillientprop
Vx = np.random.uniform(0, 1)
Vf = np.random.uniform(0, 1)

##################################################
#learning rate aplha
#learning rate for the backprop
alpha = 0.1
alphabp = alpha * 0.001


##################################################
# resillient propagation variable:
nN = 0.5
nP = 1.2
Dxinit = 0.001
Dfinit = 0.001

previousSign = 1
currentSign = 1




print(F.shape)
print(G.shape)
print(y.shape)
print(Vx)
print(Vf)

(30, 10)
(30,)
(30,)
0.7654597593969069
0.2823958439671127


In [147]:
#################################################
# useful function
def SSE(A,B):
 # E = 0.5*(((A-B)).sum())**
  E = 1 / 2 * np.sum((A - B)**2)
  return E

def SIGN(A):
  x = 0
  if A >=0:
    x = 1
  else:
    x = -1
  return x 

Forward propagation :

In [148]:
###################################################
#Forward propagation 
def forwardprop(X,y,Vx,Vf,T,I,F,G):
  for t in range(T-1):
    for i in range(I):
        F[i][t+1] = F[i][t] * Vf + X[i][t] * Vx
        G[i] = F[i][T-1] * Vf + X[i][T-1] * Vx   

            
  
  return(G)

Propagation function and gradient clipping:

In [149]:
###################################################
# backpropagation
# we have to be mindful of the difference between T 
# and in range(T) because in the second case it is starting from 0 to T-1
# so we have to adapt our notation 
def backprop(G,y,Vx,Vf,T,I,alpha):
  for t in range(T):
    for i in range(I):
      Vx = Vx - alpha * (G[i] - y[i]) * X[i][t] * (Vf**(T-(t+1)))


  for t in range(T-1):    
    for i in range(I):
      Vf = Vf - alpha *  (G[i] - y[i]) * F[i][t] * (Vf**(T-(t+1)))


      
  return(Vx,Vf)


In [150]:
#################################################
# Resillient propagation
def resillientprop(G,y,Vx,Vf,T,I,nN,nP,Dxinit,Dfinit):

#################################################
# Vx :
  Dx = Dxinit
  gradX = 0
  currentSIGNX = 1
  previousSIGNX = 1
  for t in range(T):
    for i in range(I):
        gradX = gradX +  (G[i] - y[i]) * X[i][t] * (Vf**(T-(t+1)))

  currentSIGNX = SIGN(gradX) 

  if (currentSIGNX == previousSIGNX):
    Dx = Dx * nP
  else:
    Dx = Dx * nN
# update Vx

  Vx = Vx - Dx * currentSIGNX
  previousSIGNX = currentSIGNX 

#################################################
# Vf :
  Df = Dfinit
  gradF = 0
  currentSIGNF = 1
  previousSIGNF = 1

  for t in range(T-1):    
    for i in range(I):
      gradF = gradF + (G[i] - y[i]) * F[i][t] * (Vf**(T-(t+1)))
  
  currentSIGNF = SIGN(gradF) 

  if (currentSIGNF == previousSIGNF):
    Df = Df * nP
  else:
    Df = Df * nN

# update Vf
  Vf = Vf - Df * currentSIGNF
  previousSIGNF = currentSIGNF 

  return(Vx,Vf)

In [151]:
#################################################
# gradient clipping

def gradientclip(G,y,Vx,Vf,T,I,N,alpha):
  gradX = 0
  for t in range(T):
    for i in range(I):
        gradX = gradX +  (G[i] - y[i]) * X[i][t] * (Vf**(T-(t+1)))

  if abs(gradX) > N:
    gradX = N
  
  Vx = Vx - (alpha * gradX)


  gradF = 0
  for t in range(T-1):
    for i in range(I):
        gradF = gradF +  (G[i] - y[i]) * F[i][t] * (Vf**(T-(t+1)))


  if abs(gradF) > N:
    gradF = N
  
  Vf = Vf - (alpha * gradF)

  return(Vx,Vf)

Convergence demonstration for each method using different value :

In [152]:
############################################
#RNN with backprop

# we re-initialize G and F so you can launch the other propagation 
F = np.zeros((I,T))
G = np.zeros((I))

# weight for the backprop
Vxbp = np.random.uniform(0, 1)*0.001
Vfbp = np.random.uniform(0, 1)*0.001


# the backpropagation converge for extremely small value of alpha 
#and extremely small weight initializiation
print("Error before using our RNN with backpropragation", SSE(y,G))
for i in range(2000):
  G = forwardprop(X,y,Vxbp,Vfbp,T,I,F,G)        
  (Vxbp,Vfbp) = backprop(G,y,Vxbp,Vfbp,T,I,alphabp)
print("Error after using our RNN with backpropragation",SSE(y,G))

Error before using our RNN with backpropragation 408.5
Error after using our RNN with backpropragation 2.1750764298006318


In [153]:
############################################
# RNN with resillient prop
F = np.zeros((I,T))
G = np.zeros((I))


# weight for the resillientprop
Vx = np.random.uniform(0, 1)
Vf = np.random.uniform(0, 1)

print("Error before using our RNN with resillient propragation", SSE(y,G))
for i in range(2000):
  G = forwardprop(X,y,Vx,Vf,T,I,F,G)        
  (Vx,Vf) = resillientprop(G,y,Vx,Vf,T,I,nN,nP,Dxinit,Dfinit)
print("Error after using our RNN with resillient backpropragation",SSE(y,G))

Error before using our RNN with resillient propragation 408.5
Error after using our RNN with resillient backpropragation 0.0030718243079734912


In [154]:
############################################
#RNN with gradient clipping

# we re-initialize G and F so you can launch the other propagation 
F = np.zeros((I,T))
G = np.zeros((I))

# learning rate alpha
alpha = 0.1

# value for clipping N:
N = 0.02

# weight for the backprop
Vxclip = np.random.uniform(0, 1)
Vfclip = np.random.uniform(0, 1)

# the gradient clipping converge for greater value compared to backpropagation
print("Error before using our RNN with gradient clipping", SSE(y,G))
for i in range(2000):
  G = forwardprop(X,y,Vxbp,Vfbp,T,I,F,G)        
  (Vxclip,Vfclip) = gradientclip(G,y,Vxclip,Vfclip,T,I,N,0.1)
print("Error after using our RNN with gradient clipping",SSE(y,G))

Error before using our RNN with gradient clipping 408.5
Error after using our RNN with gradient clipping 2.170657180071716


Comparaison using same initial value for the three method:

In [155]:
#############################
F = np.zeros((I,T))
G = np.zeros((I))
nN = 0.5
nP = 1.2
Dxinit = 0.001
Dfinit = 0.001
N = 0.02
alpha = 0.1 
U = np.random.uniform(0, 1)
V = np.random.uniform(0, 1)

Vxbp = U
Vfbp = V

Vx = U
Vf = V

Vxclip = np.random.uniform(0, 1)
Vfclip = np.random.uniform(0, 1)

G1 = np.zeros((I))
G2 = np.zeros((I))
G3 = np.zeros((I))

F1 = np.zeros((I,T))
F2 = np.zeros((I,T))
F3 = np.zeros((I,T))
print("Error before using our RNN with backpropragation", SSE(y,G1))
for i in range(10):
  G1 = forwardprop(X,y,Vxbp,Vfbp,T,I,F,G1)        
  (Vxbp,Vfbp) = backprop(G1,y,Vxbp,Vfbp,T,I,alpha)
print("Error after using our RNN with backpropragation",SSE(y,G1))


print("Error before using our RNN with resillient propragation", SSE(y,G2))
for i in range(2000):
  G2 = forwardprop(X,y,Vx,Vf,T,I,F,G2)        
  (Vx,Vf) = resillientprop(G2,y,Vx,Vf,T,I,nN,nP,Dxinit,Dfinit)
print("Error after using our RNN with resillient propragation",SSE(y,G2))


print("Error before using our RNN with gradient clipping", SSE(y,G3))
for i in range(2000):
  G3 = forwardprop(X,y,Vxclip,Vfclip,T,I,F,G3)        
  (Vxclip,Vfclip) = gradientclip(G3,y,Vxclip,Vfclip,T,I,N,alpha)
                    
print("Error after using our RNN with gradient clipping",SSE(y,G3))

Error before using our RNN with backpropragation 408.5
Error after using our RNN with backpropragation nan
Error before using our RNN with resillient propragation 408.5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


Error after using our RNN with resillient propragation 0.011544588094177901
Error before using our RNN with gradient clipping 408.5
Error after using our RNN with gradient clipping 1029051762172.5586


In [156]:
print("We can see that backpropagation is extremely unstable and will easily diverge")
print("We can see that gradient clipping and resillient propagation both converge with far greater value than for backpropagation")
print("But we can see that resillient progation converge way faster than gradient clipping in my example")

We can see that backpropagation is extremely unstable and will easily diverge
We can see that gradient clipping and resillient propagation both converge with far greater value than for backpropagation
But we can see that resillient progation converge way faster than gradient clipping in my example


In [157]:
G1 = np.zeros((10))
G2 = np.zeros((10))
G3 = np.zeros((10))
X = X_test
y = y_test
F = np.zeros((10,T))

print("Error before using our RNN with backpropragation", SSE(y,G1))
for i in range(10):
  G1 = forwardprop(X,y,Vxbp,Vfbp,T,10,F,G1)        
  (Vxbp,Vfbp) = backprop(G1,y,Vxbp,Vfbp,T,10,alpha)
print("Error after using our RNN with backpropragation",SSE(y,G1))

print("Error before using our RNN with resillient propragation", SSE(y,G2))
for i in range(2000):
  G2 = forwardprop(X,y,Vx,Vf,T,10,F,G2)        
  (Vx,Vf) = resillientprop(G2,y,Vx,Vf,T,10,nN,nP,Dxinit,Dfinit)
print("Error after using our RNN with resillient propragation",SSE(y,G2))

print("Error before using our RNN with gradient clipping", SSE(y,G3))
for i in range(2000):
  G3 = forwardprop(X,y,Vxclip,Vfclip,T,10,F,G3)        
  (Vxclip,Vfclip) = gradientclip(G3,y,Vxclip,Vfclip,T,10,N,alpha)
                    
print("Error after using our RNN with gradient clipping",SSE(y,G3))

Error before using our RNN with backpropragation 144.0
Error after using our RNN with backpropragation nan
Error before using our RNN with resillient propragation 144.0
Error after using our RNN with resillient propragation 0.0012534672252682278
Error before using our RNN with gradient clipping 144.0
Error after using our RNN with gradient clipping 6.348266300694062e+17


In [158]:
print(" we can see that with test value resillient gradient stay the best method we have")

 we can see that with test value resillient gradient stay the best method we have
